In [26]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_mongodb.vectorstores import MongoDBAtlasVectorSearch
from api.core.mongo import db
from langchain_core.runnables import RunnablePassthrough
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_nomic import NomicEmbeddings

C:\Users\holmt\PycharmProjects\sunhacks-2024\venv\lib\site-packages\pydantic\_internal\_fields.py:132: UserWarning: Field "model_name" in GPT4AllEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [44]:
llm = ChatOpenAI()
vector_store = MongoDBAtlasVectorSearch(
    collection=db["embeddings"], 
    embedding=NomicEmbeddings(model="nomic-embed-text-v1.5", inference_mode="local", dimensionality=768),
    index_name="vector_index", 
    relevance_score_fn="euclidean",
    text_key="text",
)

In [45]:
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


qa_chain = (
    {
        "context": vector_store.as_retriever() | format_docs,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)

C:\Users\holmt\PycharmProjects\sunhacks-2024\venv\lib\site-packages\langsmith\client.py:323: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [48]:
qa_chain.invoke("What phone can I buy? I want an 3-400$ phone.")

'You can consider purchasing the Google Pixel 4a or the iPhone SE, both of which fall within the $300-400 price range. These phones offer a good balance of features and performance for their price point. Make sure to compare their specifications and features to see which one aligns best with your needs.'